In [217]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import OneHotEncoder


In [127]:
train_ds = pd.read_csv('data/bank_data_train.csv')
test_ds = pd.read_csv('data/bank_data_test.csv')

In [145]:
test_ds['is_test'] = 1
train_ds['is_test'] = 0
df_of_all = train_ds.append(test_ds)
df_of_all

,ID,CR_PROD_CNT_IL,AMOUNT_RUB_CLO_PRC,PRC_ACCEPTS_A_EMAIL_LINK,APP_REGISTR_RGN_CODE,PRC_ACCEPTS_A_POS,PRC_ACCEPTS_A_TK,TURNOVER_DYNAMIC_IL_1M,CNT_TRAN_AUT_TENDENCY1M,SUM_TRAN_AUT_TENDENCY1M,...,MED_DEBT_PRC_YWZ,LDEAL_ACT_DAYS_PCT_TR3,LDEAL_ACT_DAYS_PCT_AAVG,LDEAL_DELINQ_PER_MAXYWZ,TURNOVER_DYNAMIC_CC_3M,LDEAL_ACT_DAYS_PCT_TR,LDEAL_ACT_DAYS_PCT_TR4,LDEAL_ACT_DAYS_PCT_CURR,TARGET,is_test
0,146841,0,0.000000,NaN,NaN,NaN,NaN,0.0,NaN,NaN,...,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,0.0,0
1,146842,0,0.041033,NaN,NaN,NaN,NaN,0.0,0.166667,0.186107,...,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,0.0,0
2,146843,0,0.006915,0.0,NaN,0.0,0.0,0.0,NaN,NaN,...,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,0.0,0
3,146844,0,0.000000,NaN,NaN,NaN,NaN,0.0,NaN,NaN,...,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,0.0,0
4,146845,0,0.000000,NaN,NaN,NaN,NaN,0.0,NaN,NaN,...,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88793,175305,0,0.000000,0.0,NaN,0.0,0.0,0.0,NaN,NaN,...,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,1
88794,275016,1,1.000000,0.0,63.0,0.0,0.0,0.0,NaN,NaN,...,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,1
88795,277398,0,0.047768,0.0,NaN,0.0,0.0,0.0,NaN,NaN,...,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,1
88796,469164,0,0.035166,NaN,NaN,NaN,NaN,0.0,1.000000,1.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,1


In [146]:
correlated_vals = s.sort_values().dropna()[-100:]
corr_cols = map(lambda i: i[1], filter(lambda i: i[0] != i[1], correlated_vals.index.values))
corr_cols_to_remove = list(corr_cols)[1:]
corr_cols_to_remove

['LDEAL_ACT_DAYS_PCT_AAVG',
 'LDEAL_ACT_DAYS_PCT_TR3',
 'LDEAL_ACT_DAYS_PCT_TR4',
 'LDEAL_ACT_DAYS_PCT_CURR',
 'LDEAL_ACT_DAYS_PCT_TR',
 'LDEAL_ACT_DAYS_PCT_CURR',
 'LDEAL_ACT_DAYS_PCT_TR']

## Filtering nan fields

In [147]:

df_of_all.drop(corr_cols_to_remove, inplace=True, axis=1)
null_fields = sorted(df_of_all.isna().sum().items(), key=lambda c: c[1], reverse=True)
fields_left = map(lambda f: f[0], filter(lambda f: f[1] < 40000, null_fields))
df_filtered = df_of_all[list(fields_left) + ['TARGET']]


In [169]:
df_filtered.fillna(0)

,id,CR_PROD_CNT_IL,TURNOVER_DYNAMIC_IL_1M,REST_DYNAMIC_FDEP_1M,REST_DYNAMIC_SAVE_3M,CR_PROD_CNT_VCU,avg_account_balance,CR_PROD_CNT_TOVR,CR_PROD_CNT_PIL,avg_turnover_credit_card,...,service_package,customer_time_months,TURNOVER_DYNAMIC_PAYM_3M,TURNOVER_DYNAMIC_PAYM_1M,REST_DYNAMIC_CC_1M,TURNOVER_DYNAMIC_CC_1M,REST_DYNAMIC_CC_3M,TURNOVER_DYNAMIC_CC_3M,target,is_test
0,146841,0,0.0,0.0,0.541683,0,156067.339767,0,0,0.0,...,K01,1.593023,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0
1,146842,0,0.0,0.0,0.000000,0,4278.845817,0,0,0.0,...,102,1.587647,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0
2,146843,0,0.0,0.0,0.000000,0,112837.062817,0,0,0.0,...,102,1.587647,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0
3,146844,0,0.0,0.0,0.005874,0,42902.902883,0,0,0.0,...,K01,1.583333,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0
4,146845,0,0.0,0.0,0.000000,0,71906.476533,0,0,0.0,...,102,1.583333,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88793,175305,0,0.0,0.0,0.000000,0,28831.108617,1,0,0.0,...,O01,10.254313,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,1
88794,275016,1,0.0,0.0,0.000000,1,563197.143400,0,0,0.0,...,K01,3.023130,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,1
88795,277398,0,0.0,0.0,0.000000,0,5206.774000,0,0,0.0,...,O01,4.098399,0.536266,0.162707,0.0,0.0,0.0,0.0,0.0,1
88796,469164,0,0.0,0.0,0.000000,0,220355.154967,1,0,0.0,...,104,6.012259,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,1


In [154]:
test_ds

,ID,CR_PROD_CNT_IL,AMOUNT_RUB_CLO_PRC,PRC_ACCEPTS_A_EMAIL_LINK,APP_REGISTR_RGN_CODE,PRC_ACCEPTS_A_POS,PRC_ACCEPTS_A_TK,TURNOVER_DYNAMIC_IL_1M,CNT_TRAN_AUT_TENDENCY1M,SUM_TRAN_AUT_TENDENCY1M,...,MED_DEBT_PRC_YWZ,LDEAL_ACT_DAYS_PCT_TR3,LDEAL_ACT_DAYS_PCT_AAVG,LDEAL_DELINQ_PER_MAXYWZ,TURNOVER_DYNAMIC_CC_3M,LDEAL_ACT_DAYS_PCT_TR,LDEAL_ACT_DAYS_PCT_TR4,LDEAL_ACT_DAYS_PCT_CURR,TARGET,is_test
0,400980,0,0.000000,NaN,NaN,NaN,NaN,0.0,NaN,NaN,...,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,1
1,525062,0,0.059381,0.0,NaN,0.0,0.0,0.0,NaN,NaN,...,1.0,0.5,0.5,0.0,0.0,0.5,0.5,0.5,NaN,1
2,280316,0,0.070430,0.0,NaN,0.0,0.0,0.0,NaN,NaN,...,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,1
3,496066,0,NaN,0.0,NaN,0.0,0.0,0.0,NaN,NaN,...,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,1
4,375031,0,0.180961,0.0,77.0,0.0,0.0,0.0,NaN,NaN,...,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88793,175305,0,0.000000,0.0,NaN,0.0,0.0,0.0,NaN,NaN,...,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,1
88794,275016,1,1.000000,0.0,63.0,0.0,0.0,0.0,NaN,NaN,...,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,1
88795,277398,0,0.047768,0.0,NaN,0.0,0.0,0.0,NaN,NaN,...,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,1
88796,469164,0,0.035166,NaN,NaN,NaN,NaN,0.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,1


In [149]:
mappings = {
   "TRANS_AMOUNT_TENDENCY3M":"transactions_sum_3_to_6_M",
   "TRANS_CNT_TENDENCY3M":"transactions_count_3_to_6_M",
   "AMOUNT_RUB_CLO_PRC":"clothes_to_all_amount",
   "AMOUNT_RUB_SUP_PRC":"supplies_to_all_amount",
   "AMOUNT_RUB_NAS_PRC":"",
   "TRANS_COUNT_SUP_PRC":"supplies_to_all_count",
   "TRANS_COUNT_NAS_PRC":"",
   "TRANS_COUNT_ATM_PRC":"Atm_to_all_count",
   "AMOUNT_RUB_ATM_PRC":"atm_to_all_amount",
   "ID":"id",
   "CR_PROD_CNT_IL":"",
   "TURNOVER_DYNAMIC_IL_1M":"",
   "REST_DYNAMIC_FDEP_1M":"",
   "REST_DYNAMIC_SAVE_3M":"",
   "CR_PROD_CNT_VCU":"",
   "REST_AVG_CUR":"avg_account_balance",
   "CR_PROD_CNT_TOVR":"",
   "CR_PROD_CNT_PIL":"",
   "TURNOVER_CC":"avg_turnover_credit_card",
   "TURNOVER_PAYM":"avg_turnover_salary_account",
   "AGE":"age_months",
   "CR_PROD_CNT_CC":"",
   "REST_DYNAMIC_FDEP_3M":"",
   "REST_DYNAMIC_IL_1M":"",
   "CR_PROD_CNT_CCFP":"",
   "REST_DYNAMIC_CUR_1M":"",
   "REST_AVG_PAYM":"avg_salary_account_balance",
   "LDEAL_GRACE_DAYS_PCT_MED":"",
   "REST_DYNAMIC_CUR_3M":"",
   "TURNOVER_DYNAMIC_CUR_1M":"",
   "REST_DYNAMIC_PAYM_3M":"",
   "REST_DYNAMIC_IL_3M":"",
   "TURNOVER_DYNAMIC_IL_3M":"",
   "REST_DYNAMIC_PAYM_1M":"",
   "TURNOVER_DYNAMIC_CUR_3M":"",
   "PACK":"service_package",
   "CLNT_SETUP_TENOR":"customer_time_months",
   "TURNOVER_DYNAMIC_PAYM_3M":"",
   "TURNOVER_DYNAMIC_PAYM_1M":"",
   "REST_DYNAMIC_CC_1M":"",
   "TURNOVER_DYNAMIC_CC_1M":"",
   "REST_DYNAMIC_CC_3M":"",
   "TURNOVER_DYNAMIC_CC_3M":"",
   "TARGET":"target",
    "is_test": "is_test"
}

In [150]:
source = []
target = []
for ms, mt in mappings.items():
    if ms not in df_filtered.columns:
        continue 
    if len(mt) == 0:
        target.append(ms)
    else:
        target.append(mt)
    source.append(ms)
df_filtered = df_filtered[source]
df_filtered.columns = target

In [151]:
df_filtered

,id,CR_PROD_CNT_IL,TURNOVER_DYNAMIC_IL_1M,REST_DYNAMIC_FDEP_1M,REST_DYNAMIC_SAVE_3M,CR_PROD_CNT_VCU,avg_account_balance,CR_PROD_CNT_TOVR,CR_PROD_CNT_PIL,avg_turnover_credit_card,...,service_package,customer_time_months,TURNOVER_DYNAMIC_PAYM_3M,TURNOVER_DYNAMIC_PAYM_1M,REST_DYNAMIC_CC_1M,TURNOVER_DYNAMIC_CC_1M,REST_DYNAMIC_CC_3M,TURNOVER_DYNAMIC_CC_3M,target,is_test
0,146841,0,0.0,0.0,0.541683,0,156067.339767,0,0,0.0,...,K01,1.593023,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0
1,146842,0,0.0,0.0,0.000000,0,4278.845817,0,0,0.0,...,102,1.587647,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0
2,146843,0,0.0,0.0,0.000000,0,112837.062817,0,0,0.0,...,102,1.587647,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0
3,146844,0,0.0,0.0,0.005874,0,42902.902883,0,0,0.0,...,K01,1.583333,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0
4,146845,0,0.0,0.0,0.000000,0,71906.476533,0,0,0.0,...,102,1.583333,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88793,175305,0,0.0,0.0,0.000000,0,28831.108617,1,0,0.0,...,O01,10.254313,0.000000,0.000000,0.0,0.0,0.0,0.0,NaN,1
88794,275016,1,0.0,0.0,0.000000,1,563197.143400,0,0,0.0,...,K01,3.023130,0.000000,0.000000,0.0,0.0,0.0,0.0,NaN,1
88795,277398,0,0.0,0.0,0.000000,0,5206.774000,0,0,0.0,...,O01,4.098399,0.536266,0.162707,0.0,0.0,0.0,0.0,NaN,1
88796,469164,0,0.0,0.0,0.000000,0,220355.154967,1,0,0.0,...,104,6.012259,0.000000,0.000000,0.0,0.0,0.0,0.0,NaN,1


In [228]:
test = df_filtered.query('is_test == 1')
train = df_filtered.query('is_test == 0')

X_train, X_test, y_train, y_test = train_test_split(train.fillna(0).drop('target', axis=1), train.target, test_size=0.2)
X_train[col].values
for col in X_train.columns:
    lr = LogisticRegression()
    print(f'col: {col}')
#     print(X_train[col].values.reshape(-1, 1))
    lr.fit(X_train[col].values.reshape(-1, 1), y_train)
    r_auc = roc_auc_score(X_test[col].values.reshape(-1, 1), y_test, multi_class='ovr')
    print(f'score: {r_auc}')
#     print(sum(lr.predict(X_test[col].values.reshape(-1, 1))))


    

col: id


AxisError: axis 1 is out of bounds for array of dimension 1

In [223]:
OneHotEncoder().fit(y_test)

ValueError: Expected 2D array, got 1D array instead:
array=[0. 0. 0. ... 0. 0. 0.].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.

In [224]:
OneHotEncoder?

In [210]:
len(train)

355190

In [214]:
roc_auc_score??